# AUTOGEN MULTI-AGENT ORCHESTRATION

AutoGen's multi-agent orchestration enables seamless collaboration between AI agents, allowing them to autonomously delegate tasks, retrieve information, and execute complex workflows. By leveraging structured communication, tool integration, and intelligent decision-making, AutoGen facilitates scalable AI-driven applications across domains like research, automation, and customer support. With flexible orchestration models such as RoundRobin, SelectorGroupChat, and Swarm, developers can design adaptive multi-agent ecosystems that efficiently coordinate and optimize task execution.

Using multiple agents in AI Automation have the following benefits:
- **Specialization:** Instead of one massive model trying to handle everything (translation, coding, web-surfing, etc.), you can compose smaller, specialized agents.
- **Parallelism & Modularity:** Multiple agents can divide tasks or work in parallel without stepping on each other’s toes.
- **Scalable Expertise:** You can plug in new agents (like a VideoSurfer, FileSurfer, or SQLQueryAgent) without rewriting existing logic.


New requirements to manage the complexity of using multiple agents...
- **Orchestration:** If multiple agents can respond at once, you need a coordinating mechanism (how do they know who speaks next? do they pass tasks around?). AutoGen provides RoundRobinGroupChat, SelectorGroupChat, Swarm, and MagenticOneGroupChat for this orchestration.
- **Shared Context & State:** Agents must see the most recent conversation state and outputs of other agents. AutoGen’s team concept ensures agents share a single conversation thread or message buffer.
- **Tool Integration:** Each agent might call external APIs. If they conflict, you need to define a tool schema or allow parallel/discrete usage. In AutoGen, you can pass different tools or function-call endpoints to each agent. That way, the finance agent might have a currency API tool, while a web-search agent has a search tool.
- **Memory & Persistence:** Long-running multi-agent applications need a memory store to recall user preferences, partial results, or conversation states across steps. AutoGen’s Memory interface (e.g., ListMemory, vector-based memory) can store facts so that all agents can retrieve relevant knowledge.-

AutoGen’s **AgentChat** is a framework that enables multi-agent interactions in a structured, conversational manner. It provides a set of preset orchestration patterns (e.g., RoundRobinGroupChat, SelectorGroupChat, etc.) as well as tools to build custom multi-agent collaboration. Each agent is typically backed by a language model and may have specific tools or roles (e.g., “critic,” “coder,” “researcher”), and AgentChat ensures these agents share conversation state, handle tool calls, and stop under specified termination conditions.

In AutoGen’s AgentChat, when you want multiple agents to collaborate in one conversation, you typically put them in a team. A **“team”** is the mechanism that:

- Combines multiple agents (each with its own role or specialty) under one conversation.
- Manages how the agents take turns, share context, and optionally call external tools.
- Enforces any termination or concurrency rules you set (e.g., stopping after 10 messages).

Whether you choose RoundRobinGroupChat, SelectorGroupChat, Swarm, or MagenticOneGroupChat, you’re effectively creating a team object that orchestrates multiple agents. This ensures they don’t talk over one another or lose track of the conversation state. If you only have one agent, you can run it by itself—no team required. But once you have more than one, a team is always the recommended structure to keep the interactions synchronized.


## Example1: Multi-Agent Orchestration with RoundRobinGroupChat

Below is a minimal multi-agent orchestration example using Azure OpenAI. We create two agents—one with multiple tools, and a second “critic” agent—then place them in a RoundRobinGroupChat so they can take turns responding. This setup demonstrates how to integrate Azure OpenAI, load environment variables, define tool functions, and orchestrate more than one agent in a conversation.

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

# AutoGen imports for agents, teams, and console UI
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Azure OpenAI client
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# 1) Load environment variables from .env (containing endpoint & API key)
load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

# 2) Create an AzureOpenAIChatCompletionClient
azure_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# 3) Define two agents:
#    - QuantumTeacher: explains quantum superposition
#    - SkepticalStudent: asks clarifying questions
quantum_teacher = AssistantAgent(
    name="QuantumTeacher",
    model_client=azure_client,
    system_message="You are a physics teacher explaining quantum superposition in simple terms."
)

skeptical_student = AssistantAgent(
    name="SkepticalStudent",
    model_client=azure_client,
    system_message="You are a curious student with follow-up questions on quantum superposition."
)

# 4) RoundRobinGroupChat ensures the two agents take turns automatically
team = RoundRobinGroupChat(
    participants=[quantum_teacher, skeptical_student]
)

# 5) In a Jupyter notebook cell, just await the run_stream
await Console(team.run_stream(task="Explain quantum superposition in simple terms."))


---------- user ----------
Explain quantum superposition in simple terms.
---------- QuantumTeacher ----------
Sure! Quantum superposition is a basic idea in quantum mechanics, which is the branch of physics that deals with very tiny particles, like atoms and subatomic particles.

Imagine you have a coin. When you flip it, it can land on heads or tails. But right when you flip it, while it's spinning in the air, you can think of it as being in a superposition of both heads and tails at the same time. It’s only when you catch it and look at it that it "decides" to become either heads or tails.

In the quantum world, particles, like electrons, can be in multiple states at the same time, just like the spinning coin. For example, an electron can be in two different places at once or have different energy levels simultaneously. It’s only when we measure or observe the particle that it settles into one specific state—like when you catch the coin and see whether it’s heads or tails.

This ide

The example highlights how **RoundRobinGroupChat** ensures each agent speaks in turn and broadcasts its message so the other agent can see the context. Specifically:

- Predictable Turn-Taking: The conversation always alternates between QuantumTeacher and  SkepticalStudent, rather than letting them talk over each other.
- Shared Context: When QuantumTeacher provides an explanation, SkepticalStudent sees that text before responding with questions, ensuring a coherent back-and-forth conversation.

Ensuring predictable turn-taking and shared context keeps multi-agent conversations organized and coherent. Each agent speaks in a well-defined order, preventing them from “talking over” one another, while having every agent see the entire history helps avoid contradictions or missing references. However, if concurrency is needed or certain content is private, strict round-robin and full visibility can become problematic—unnecessary turns may slow progress, and sensitive details might get exposed to agents that don’t need them.

## Example2: Multi-Agent Orchestration with SelectorGroupChat and different LLM's

Below is a minimal example of using SelectorGroupChat, where a model decides which agent should reply next. Each agent specializes in a different domain (math, translation, philosophy), and the team’s selector logic picks the most relevant agent for each user question.

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

# AutoGen imports for multi-agent chat
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

# Azure OpenAI clients
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# 1) Load environment variables for Azure config (endpoint, API key)
load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT", "")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY", "")

# 2) Define two AzureOpenAIChatCompletionClients:
#    (a) aggregator_client (larger gpt-4o), used by the SELECTOR
#    (b) agent_client (smaller gpt-4o-mini), used by each AGENT

aggregator_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o",
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

agent_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

# 3) Define three specialized agents (all using the smaller model)
math_agent = AssistantAgent(
    name="MathAgent",
    model_client=agent_client,
    system_message="You solve math queries. Only handle math questions."
)

translator_agent = AssistantAgent(
    name="TranslatorAgent",
    model_client=agent_client,
    system_message="You translate phrases between languages. Only handle translation requests."
)

philosopher_agent = AssistantAgent(
    name="PhilosopherAgent",
    model_client=agent_client,
    system_message="You provide thoughtful philosophical insights. Only handle philosophical queries."
)

# 4) Provide a selector prompt for deciding which agent responds
selector_prompt = """You are the orchestrator using GPT-4o. 
Review the conversation context, then return exactly one role from: 
MathAgent, TranslatorAgent, PhilosopherAgent.
"""

# 5) Create the SelectorGroupChat with aggregator_client for the SELECTOR
team = SelectorGroupChat(
    participants=[math_agent, translator_agent, philosopher_agent],
    model_client=aggregator_client,   # <-- Larger Azure GPT-4 for selecting next speaker
    selector_prompt=selector_prompt,
    allow_repeated_speaker=False
)

# 6) Jupyter-friendly code for streaming the conversation
await Console(team.run_stream(
    task="1) Solve 45*32 2) Translate 'Hello' to Spanish 3) What's the nature of reality?"
))


---------- user ----------
1) Solve 45*32 2) Translate 'Hello' to Spanish 3) What's the nature of reality?
---------- MathAgent ----------
1) 45 * 32 = 1440.
